In [27]:
# Libraries
import os, sys

%reload_ext autoreload
%autoreload 2
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [28]:
from app.nqueens import NQueensGeneticAlgorithm

nQueensGA = NQueensGeneticAlgorithm(population_size=100, dimension=8)

nQueensGA.run(
    generations=500,
    mutation_probability=0.02,
    crossover_probability=0.8,
    mutation_operator="inversion_mutation",
    crossover_operator="single_cross",
    selection_operator="tournament_selection"
)

nQueensGA.report()

{'generations': 500,
 'duration': '0:00:00.258086',
 'best_fitness': 28,
 'best_fitness_percentage': 100.0,
 'best_representation': [5, 3, 0, 4, 7, 1, 6, 2],
 'worst_fitness': 28,
 'worst_representation': [5, 3, 0, 4, 7, 1, 6, 2],
 'mean_fitness': 28.0,
 'selection_operator': 'tournament_selection',
 'mutate_operator': 'inversion_mutation',
 'crossover_operator': 'single_point_co'}